# AutoMM for Named Entity Recognition - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/multimodal/text_prediction/ner.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/multimodal/text_prediction/ner.ipynb)

Named entity recognition (NER) refers to identifying and categorizing key information (entities) from unstructured text. An entity can be a word or a series of words which correspond to categories such as cities, time expressions, monetary values, facilities, person, organization, etc. An NER model usually takes as input an unannotated block of text and output an annotated block of text that highlights the named entities with predefined categories. For example, given the following sentences,

- Albert Einstein was born in Germany and is widely acknowledged to be one of the greatest physicists.

The model will tell you that "Albert Einstein" is a PERSON and "Germany" is a LOCATION. In the following, we will introduce how to use AutoMM for the NER task, including how to prepare your data, how to train your model, and what you can expect from the model outputs.


## Prepare Your Data
Like other tasks in AutoMM, all you need to do is to prepare your data as data tables (i.e., dataframes) which contain a text column and an annotation column. The text column stores the raw textual data which contains the entities you want to identify. Correspondingly, the annotation column stores the label information (e.g., the *category* and the *start/end* offset in character level) for the entities. AutoMM requires the *annotation column* to
have the following json format (Note: do not forget to call json.dumps() to convert python objects into a json string before creating your dataframe).

In [1]:
!pip install autogluon.multimodal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 842.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtim

In [1]:
!pip uninstall torchvision -y
!pip install torchvision

Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
ERROR: pip's dependency resolver does not currently

In [2]:
import json
json.dumps([
    {"entity_group": "PERSON", "start": 0, "end": 15},
    {"entity_group": "LOCATION", "start": 28, "end": 35}
])

'[{"entity_group": "PERSON", "start": 0, "end": 15}, {"entity_group": "LOCATION", "start": 28, "end": 35}]'

where **entity_group** is the category of the entity and **start** is a character-level position indicating where the entity begins while **end** represents the ending position of the entity. To make sure that AutoMM can recognise your json annotations, it is required to use the exactly same keys/properties (entity_group, start, end) specified above when constructing your data. You can annote "Albert Einstein" as a single entity group or you can also assign each word a label.

Following is an example of visualizing the annotations with the `visualize_ner` utility.

In [3]:
from autogluon.multimodal.utils import visualize_ner

sentence = "Albert Einstein was born in Germany and is widely acknowledged to be one of the greatest physicists."
annotation = [{"entity_group": "PERSON", "start": 0, "end": 15},
              {"entity_group": "LOCATION", "start": 28, "end": 35}]

visualize_ner(sentence, annotation)

If you are already familiar with the NER task, you probably have heard about the [BIO](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) (Benginning-Inside-Outside) format. You can adopt this format (which is not compulsory) to add an *I-prefix* or a *B-prefix* to each tag to indicate wether the tag is the beginning of the annotated chunk or inside the chunk. For example, you can annotate "Albert" as "B-PERSON" because it is the beginning of the name and "Einstein" as "I-PERSON" as it is inside the PERSON chunk. You do not need to worry about the *O* tags, an *O* tag indicates that a word belongs to no chunk, as AutoMM will take care of that automatically.

Now, let's look at an example dataset. This dataset is converted from the [MIT movies corpus](https://groups.csail.mit.edu/sls/downloads/movie/) which provides annotations on entity groups such as actor, character, director, genre, song, title, trailer, year, etc.

In [4]:
from autogluon.core.utils.loaders import load_pd
train_data = load_pd.load('https://automl-mm-bench.s3.amazonaws.com/ner/mit-movies/train_v2.csv')
test_data = load_pd.load('https://automl-mm-bench.s3.amazonaws.com/ner/mit-movies/test_v2.csv')
train_data.head(5)

,text_snippet,entity_annotations
0,what movies star bruce willis,"[{""entity_group"": ""ACTOR"", ""start"": 17, ""end"":..."
1,show me films with drew barrymore from the 1980s,"[{""entity_group"": ""ACTOR"", ""start"": 19, ""end"":..."
2,what movies starred both al pacino and robert ...,"[{""entity_group"": ""ACTOR"", ""start"": 25, ""end"":..."
3,find me all of the movies that starred harold ...,"[{""entity_group"": ""ACTOR"", ""start"": 39, ""end"":..."
4,find me a movie with a quote about baseball in it,[]


Let's print a row.

In [5]:
print(f"text_snippet: {train_data['text_snippet'][1]}")
print(f"entity_annotations: {train_data['entity_annotations'][1]}")
visualize_ner(train_data['text_snippet'][1], train_data['entity_annotations'][1])

text_snippet: show me films with drew barrymore from the 1980s
entity_annotations: [{"entity_group": "ACTOR", "start": 19, "end": 33}, {"entity_group": "YEAR", "start": 43, "end": 48}]


## Training
Now, let's create a predictor for named entity recognition by setting the *problem_type* to **ner** and specifying the label column. Afterwards, we call predictor.fit() to train the model for five minutes. To achieve reasonable performance in your applications, you are recommended to set a longer enough time_limit (e.g., 30/60 minutes). You can also specify your backbone model and other hyperparameters using the hyperparameters argument. Here, we save the model to the directory `"automm_ner"`. For the purpose of demonstration, we use the lightweighted `'google/electra-small-discriminator'` backbone.

In [7]:
from autogluon.multimodal import MultiModalPredictor
import uuid

label_col = "entity_annotations"
model_path = f"./tmp/{uuid.uuid4().hex}-automm_ner"  # You can rename it to the model path you like
predictor = MultiModalPredictor(problem_type="ner", label=label_col, path=model_path)
predictor.fit(
    train_data=train_data,
    hyperparameters={'model.ner_text.checkpoint_name':'google/electra-small-discriminator'},
    time_limit=1000, #second
)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.4.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       9.00 GB / 12.67 GB (71.0%)
Disk Space Avail:   61.40 GB / 107.72 GB (57.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/tmp/904b9292e45d4bb7a5aee58e3a372e19-automm_ner
    ```

INFO: Seed set to 0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:16:41. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 5: 'val_ner_token_f1' reached 0.03185 (best 0.03185), saving model to '/content/tmp/904b9292e45d4bb7a5aee58e3a372e19-automm_ner/epoch=0-step=5.ckpt' as top 3
/usr/local/lib/python3.10/dist-packages/autogluon/multimodal/utils/checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use ca

## Evaluation
Evaluation is also straightforward, we use [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) for NER evaluation and the supported metrics are *overall_recall*, *overall_precision*, *overall_f1*, *overall_accuracy*. If you are interested in seeing the performance on a specific entity group, you can use the entity group name as the evaluation metric with which you will obtain the performances (precision, recall, f1) on the given entity group:

In [8]:
predictor.evaluate(test_data,  metrics=['overall_recall', "overall_precision", "overall_f1", "actor"])

Predicting: |          | 0/? [00:00<?, ?it/s]

{'overall_recall': 0.02753324592620341,
 'overall_precision': 0.00683911789336559,
 'overall_f1': 0.010956657846681326,
 'actor': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 812}}

## Prediction + Visualization
You can easily obtain the predictions given an input sentence by by calling predictor.predict(). If you are running the code in a Jupyter notebook, you can also easily visualize the predictions using the `visualize_ner` function which will highlight the named entities and their labels in a text.

In [9]:
from autogluon.multimodal.utils import visualize_ner

sentence = "Game of Thrones is an American fantasy drama television series created by David Benioff"
predictions = predictor.predict({'text_snippet': [sentence]})
print('Predicted entities:', predictions[0])

# Visualize
visualize_ner(sentence, predictions[0])

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicted entities: [{'entity_group': 'TRAILER', 'start': 0, 'end': 4}, {'entity_group': 'DIRECTOR', 'start': 5, 'end': 7}, {'entity_group': 'RATINGS_AVERAGE', 'start': 8, 'end': 15}, {'entity_group': 'DIRECTOR', 'start': 16, 'end': 18}, {'entity_group': 'RATING', 'start': 19, 'end': 21}, {'entity_group': 'SONG', 'start': 22, 'end': 30}, {'entity_group': 'TRAILER', 'start': 31, 'end': 38}, {'entity_group': 'RATING', 'start': 39, 'end': 44}, {'entity_group': 'YEAR', 'start': 45, 'end': 55}, {'entity_group': 'TITLE', 'start': 56, 'end': 62}, {'entity_group': 'RATING', 'start': 63, 'end': 70}, {'entity_group': 'SONG', 'start': 71, 'end': 73}, {'entity_group': 'YEAR', 'start': 74, 'end': 79}, {'entity_group': 'YEAR', 'start': 80, 'end': 87}]


## Prediction Probabilities

You can also output the probabilities for a deep dive into the predictions.

In [10]:
predictions = predictor.predict_proba({'text_snippet': [sentence]})
print(predictions[0][0]['probability'])

Predicting: |          | 0/? [00:00<?, ?it/s]

{'O': 0.022274086, 'I-YEAR': 0.06953434, 'I-CHARACTER': 0.002748113, 'B-SONG': 0.004722761, 'B-YEAR': 0.021846604, 'B-RATING': 0.02791241, 'I-RATINGS_AVERAGE': 0.0054143695, 'B-CHARACTER': 0.0027787348, 'B-DIRECTOR': 0.0061633466, 'I-TITLE': 0.03415147, 'B-TITLE': 0.012207981, 'I-TRAILER': 0.29769525, 'I-REVIEW': 0.0027243875, 'I-SONG': 0.022923302, 'I-RATING': 0.024880309, 'B-ACTOR': 0.091376185, 'B-PLOT': 0.014625063, 'B-TRAILER': 0.06666888, 'B-REVIEW': 0.00071447954, 'I-GENRE': 0.038317915, 'B-GENRE': 0.0069834515, 'I-ACTOR': 0.0038267865, 'B-RATINGS_AVERAGE': 0.015207398, 'I-DIRECTOR': 0.17715265, 'I-PLOT': 0.005837571}


## Reloading and Continuous Training
The trained predictor is automatically saved and you can easily reload it using the path. If you are not saftisfied with the current model performance, you can continue training the loaded model with new data.

In [11]:
new_predictor = MultiModalPredictor.load(model_path)
new_model_path = f"./tmp/{uuid.uuid4().hex}-automm_ner_continue_train"
new_predictor.fit(train_data, time_limit=60, save_path=new_model_path)
test_score = new_predictor.evaluate(test_data, metrics=['overall_f1', 'ACTOR'])
print(test_score)

Load pretrained checkpoint: /content/tmp/904b9292e45d4bb7a5aee58e3a372e19-automm_ner/model.ckpt
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.4.1+cu121
CUDA Version:       CUDA is not available
Memory Avail:       7.83 GB / 12.67 GB (61.8%)
Disk Space Avail:   61.35 GB / 107.72 GB (57.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/tmp/b6adea43f2b24886a7093d90efd03d70-automm_ner_continue_train
    ```

INFO: Seed set to 0
GPU Count: 0
GPU Count to be Used: 0

INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: 
  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Time limit reached. Elapsed time is 0:01:07. Signaling Trainer to stop.


Validation: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/tmp/b6adea43f2b24886a7093d90efd03d70-automm_ner_continue_train")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




Predicting: |          | 0/? [00:00<?, ?it/s]

{'overall_f1': 0.010956657846681326, 'ACTOR': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 812}}


## Other Examples

You may go to [AutoMM Examples](https://github.com/autogluon/autogluon/tree/master/examples/automm) to explore other examples about AutoMM.

## Customization
To learn how to customize AutoMM, please refer to [Customize AutoMM](../advanced_topics/customization.ipynb).